In [57]:
import pandas as pd
import datetime as dt
from  nltk import bigrams,trigrams
from gensim.corpora.dictionary import Dictionary
from gensim import  models
import re
pd.set_option('display.max_colwidth', -1)

In [55]:
#Parameters
# value between 0.1 to 1. filter days when there were fewer tweets than this quantile
quantile_filter = 0.5
num_topics=2
filter_words_that_appeared_less_than =5
use_bigrams_phrase =True
use_trigrams_phrase = True
remove_user_name=True

In [5]:
#load tweets
tweets = pd.read_csv('tweets_israeli-girls.csv')
#set 'Text column as text
tweets['Text'] = tweets['Text'].astype(str)
#set text column as datetime
tweets['Date Created'] = pd.to_datetime(tweets['Date Created'])

In [6]:
#Looking at tweets between srart and end data that contains specific word
start_date = dt.datetime(2019,1,29)
end_date = start_date +dt.timedelta(days=1)
tweets[tweets['Date Created'].between(start_date,end_date)][tweets['Text'].str.contains("גנץ")]

C:\Users\ortal\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,ID,User ID,Date Created,Text,Source,Language,Favorites,Retweets
3218,1090271744156487680,1077220361119383554,2019-01-29 15:33:34,"RT @radio103fm: השר @KahlonMoshe ל-@BenCaspit ואריה אלדד: ""איפה גנץ? לא יודע, הוא צריך להגיד הערב. תפקידים גדולים כמו שר אוצר וראש ממשלה דו…","<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",iw,NaN,1.0
3228,1090271425821200385,1077218334322683905,2019-01-29 15:32:18,RT @MakorRishon: מה הצבע האמיתי של העיניים של גנץ? ⬅️ https://t.co/NM5nvf8Klx,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",iw,NaN,3.0
3434,1090277988288159752,1077203225420402688,2019-01-29 15:58:23,RT @MaarivOnline: אבי גבאי: אני מברך איחודים הם מבורכים גנץ ויעלון צריכים להחליט האם הם מוכנים להמשיך להנציח את שלטון נתניהו או להחליט לנצח…,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",iw,NaN,2.0
3750,1090178979741532160,1077210845824847872,2019-01-29 09:24:57,"RT @ynetalerts: ליברמן מגבה את גנץ: ""נתניהו הוא זה שאחראי על צוק איתן""\nhttps://t.co/jejLk8OEQ9 https://t.co/0qw5wYgte2","<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",iw,NaN,3.0
4342,1090352828693037057,1077212969040252940,2019-01-29 20:55:46,"RT @newsisrael13: השרה @regev_miri הגיבה לנאומו של בני גנץ: ""בני גנץ הוא השמאל החדש. נאום מלא סיסמאות ריקות מתוכן"" @akivanovick https://t.c…","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",iw,NaN,5.0
4344,1090276968363425792,1077212969040252940,2019-01-29 15:54:20,"RT @newyamin: תגובתנו לאיחוד גנץ/בוגי:\nאדריכלי התיקו, גנץ ויעלון, התאחדו.\nפעם הקודמת שבוגי וגנץ היו יחד זה נגמר ב30 מנהרות טרור לדרום.\nשניה…","<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",iw,NaN,18.0
4438,1090354802205384706,1077208008592105473,2019-01-29 21:03:37,"RT @BarakRavid: זה לא בני גנץ, זה יצחק רבין","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",iw,NaN,28.0
4439,1090284182776045569,1077208008592105473,2019-01-29 16:23:00,"יעלון על החיבור עם גנץ: ""צריך לייצר כוח שינצח בבחירות. לא אכנס לממשלה בראשות נתניהו","<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",iw,NaN,NaN
4522,1090278106114469888,1077215393415090178,2019-01-29 15:58:51,"RT @swid4: בוגי, צבי האוזר, יועז הנדל. כולם אנשים טובים וראויים, אבל צובעים את מפלגת גנץ בצבע ברור. זו לא אלטרנטיבה לימין. עכשיו ברור יותר…","<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",iw,NaN,16.0
4604,1090272089485889537,1077208524260868096,2019-01-29 15:34:57,"RT @radio103fm: ""לא רואה את הבשורה בצירוף @bogie_yaalon לבני גנץ""- @RavivDrucker ל-@BenCaspit ואריה אלדד: ""בוגי עלול להיות נטל מדיני ומקטין…","<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",iw,NaN,2.0


In [8]:
#add column only the hour of 'Date Created'
hours = []
for d in tweets['Date Created']:
    hours.append(d.hour)
tweets['Hour'] = hours

In [9]:
# find tweets min and max 'Date Created'
max_date =  tweets['Date Created'].max()
max_date_limit = dt.datetime(max_date.year,max_date.month,max_date.day)+dt.timedelta(days=1)
min_date = tweets['Date Created'].min()
min_date_limit = dt.datetime(min_date.year,min_date.month,min_date.day)

In [51]:
dic_tweet_per_day = dict()
start_date =min_date_limit 
while (start_date<max_date_limit):
    end_date = start_date+dt.timedelta(days=1)
    current = tweets[tweets['Date Created'].between(start_date,end_date)] 
    if len(current)>0:
        dic_tweet_per_day[start_date] =current
    start_date = end_date   

In [32]:
dic_counts = {'Date':[], 'Count':[]}
for k in dic_tweet_per_day.keys():
    dic_counts['Date'].append(k) 
    count =len(dic_tweet_per_day[k])
    dic_counts['Count'].append(count)

count_df = pd.DataFrame(dic_counts)
filter_value = count_df['Count'].quantile(quantile_filter)

In [29]:
print(count_df['Count'].quantile(.5))

157.0


In [28]:
 count_df['Count'].median()

157.0

In [17]:
with open("./stopwords.txt", encoding="utf-8") as f:
    stopwords = set(f.readlines())
    stopwords = {w.replace("\n", "") for w in stopwords}

In [65]:
def preprocess(input_str):
    PUNCTUATION = {",", ".", "!", "?", ";", "-", "*", "&", "|",":" , '(', ')'}
    result = input_str
    #remove links
    result = " ".join([t for t in result.split() if not t.startswith("http")])
    mentions = re.findall("@[a-zA-Z0-9_.]*", result)
    #remove user name (@UserName)
    if remove_user_name:
        for mention in mentions:
            result = result.replace(mention, "")
    #remove punctuation
    for sign in PUNCTUATION:
        result = result.replace(sign, " ")
    #remove '
    result = result.replace("'", "")
    #remove stopwords
    for stopword in stopwords:
        result = result.replace(" {}".format(stopword), "")
        result = result.replace("{} ".format(stopword), "")
    #create unigram, bigrams and trigrams
    unigram = [w for w in result.split() if len(w)>1]
    features = unigram
    if use_bigrams_phrase:
        bigrams_phrase = [b[0]+" "+b[1] for b in bigrams(unigram)] 
        features += bigrams_phrase
    if use_trigrams_phrase:
        trigrams_phrase =[b[0]+" "+b[1]+" "+b[2] for b in trigrams(unigram)]
        features += trigrams_phrase
    return features

In [66]:
dic_for_data_fram = {'Date':[], 'Words':[]}

dic_suspicion=dict()
start_date =min_date_limit 
while (start_date<max_date_limit):
    end_date = start_date+dt.timedelta(days=1)
    current = tweets[tweets['Date Created'].between(start_date,end_date)] 
    if len(current)>=filter_value:
        df = current

        processed_docs = df['Text'].map(preprocess)

        dictionary = Dictionary(processed_docs)
        dictionary.filter_extremes(no_below=filter_words_that_appeared_less_than,keep_n=100000)
        
        bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
        
        if len(dictionary)>0:
            print(start_date)
            lda_model = models.LdaMulticore(bow_corpus, num_topics=num_topics, id2word=dictionary, passes=2, workers=2)

            for idx, topic in lda_model.print_topics(-1):
                print(topic)
                dic_for_data_fram['Date'].append(start_date)
                dic_for_data_fram['Words'].append(topic)

    dic_tweet_per_day[start_date] =current
    start_date = end_date

2018-12-30 00:00:00
0.175*"gt" + 0.090*"gt gt" + 0.069*"בית" + 0.057*"במכתב" + 0.057*"החוק" + 0.056*"בחירות" + 0.053*"קוריאה" + 0.051*"בני" + 0.051*"#מהבוער" + 0.049*"פוטין"
0.129*"בנט" + 0.076*"אחוז" + 0.072*"החדשה" + 0.070*"תייגו" + 0.062*"השנה" + 0.062*"ושקד" + 0.054*"בנט ושקד" + 0.053*"בכנסת" + 0.050*"נתניהו" + 0.050*"ארה"ב"
2018-12-31 00:00:00
0.138*"עוז" + 0.117*"עמוס" + 0.084*"עמוס עוז" + 0.080*"#HappyNewYear2019" + 0.059*"שנה" + 0.042*"לעמוס עוז" + 0.042*"לעמוס" + 0.042*"השנה" + 0.042*"בית" + 0.041*"ח"כ"
0.084*"גלנט" + 0.066*"צה"ל" + 0.060*"השר" + 0.049*"חיל" + 0.048*"ראש" + 0.048*"האוויר" + 0.048*"חיל האוויר" + 0.043*"לחוק" + 0.037*"שהוטס בניגוד לחוק" + 0.037*"מעצרו"
2019-01-01 00:00:00
0.096*"יו"ר" + 0.072*"הציוני" + 0.053*"שני" + 0.052*"2018" + 0.052*"ועדת" + 0.042*"בימים" + 0.042*"באופן" + 0.041*"ראש" + 0.037*"המחנה" + 0.036*"המחנה הציוני"
0.140*"לבני" + 0.071*"ציפי" + 0.059*"אבי" + 0.053*"מפלגת" + 0.053*"העבודה" + 0.053*"ציפי לבני" + 0.044*"הציוני" + 0.039*"מפלגת העבודה" +

In [67]:
pd.DataFrame(dic_for_data_fram).to_csv('./suspicion_bots_topics.csv',index=False)